In [1]:
import pandas as pd
import numpy as np

In [2]:
place=pd.read_csv("place_data.csv")
place.head(2)

,Unnamed: 0,이름,코드,태그
0,0,고른햇살,Num=a_nam_0002,#먹어야산다#분식#정대후문#포장#혼밥#가성비
1,1,영철버거,Num=a_nam_0041,#먹어야산다#이공계#경양식#버거#맥주#치킨#가성비#저렴#포장


In [3]:
place.shape

(465, 4)

In [4]:
type(place['태그'])

pandas.core.series.Series

In [5]:
place['tag']=place['태그'].copy()
place.head(2)

,Unnamed: 0,이름,코드,태그,tag
0,0,고른햇살,Num=a_nam_0002,#먹어야산다#분식#정대후문#포장#혼밥#가성비,#먹어야산다#분식#정대후문#포장#혼밥#가성비
1,1,영철버거,Num=a_nam_0041,#먹어야산다#이공계#경양식#버거#맥주#치킨#가성비#저렴#포장,#먹어야산다#이공계#경양식#버거#맥주#치킨#가성비#저렴#포장


In [6]:
place['tag'][0]

'#먹어야산다#분식#정대후문#포장#혼밥#가성비'

In [7]:
#tag를 '#'기준으로 split

place['tag']=place['tag'].str.split(pat="#")
place.head(2)

,Unnamed: 0,이름,코드,태그,tag
0,0,고른햇살,Num=a_nam_0002,#먹어야산다#분식#정대후문#포장#혼밥#가성비,"[, 먹어야산다, 분식, 정대후문, 포장, 혼밥, 가성비]"
1,1,영철버거,Num=a_nam_0041,#먹어야산다#이공계#경양식#버거#맥주#치킨#가성비#저렴#포장,"[, 먹어야산다, 이공계, 경양식, 버거, 맥주, 치킨, 가성비, 저렴, 포장]"


In [8]:
#tag를 띄어쓰기 기준으로 붙이기

place['tag']=place['tag'].apply(lambda x: " ".join(x))
place.head(2)

,Unnamed: 0,이름,코드,태그,tag
0,0,고른햇살,Num=a_nam_0002,#먹어야산다#분식#정대후문#포장#혼밥#가성비,먹어야산다 분식 정대후문 포장 혼밥 가성비
1,1,영철버거,Num=a_nam_0041,#먹어야산다#이공계#경양식#버거#맥주#치킨#가성비#저렴#포장,먹어야산다 이공계 경양식 버거 맥주 치킨 가성비 저렴 포장


In [9]:
place.tag.head(2)

0              먹어야산다 분식 정대후문 포장 혼밥 가성비
1     먹어야산다 이공계 경양식 버거 맥주 치킨 가성비 저렴 포장
Name: tag, dtype: object

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
# tag에 대해서 tf-idf 수행
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(place.tag)
print(tfidf_matrix.shape)

(465, 374)


In [12]:
#코사인 유사도
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [13]:
cosine_sim.shape

(465, 465)

In [14]:
cosine_sim

array([[1.        , 0.1829687 , 0.32195165, ..., 0.17059754, 0.02249117,
        0.01932993],
       [0.1829687 , 1.        , 0.07223991, ..., 0.09000334, 0.01186583,
        0.01019803],
       [0.32195165, 0.07223991, 1.        , ..., 0.09322109, 0.01229005,
        0.01056262],
       ...,
       [0.17059754, 0.09000334, 0.09322109, ..., 1.        , 0.13183762,
        0.11330718],
       [0.02249117, 0.01186583, 0.01229005, ..., 0.13183762, 1.        ,
        0.01493815],
       [0.01932993, 0.01019803, 0.01056262, ..., 0.11330718, 0.01493815,
        1.        ]])

In [15]:
#인덱스 테이블 만들기
#인덱스가 코드이고 인덱스 번호가 값인 판다스 시리즈 배열 선언. 
#코드 입력하면 그에 해당하는 인덱스 번호를 추출

indices = pd.Series(place.index, index=place.코드).drop_duplicates()
print(indices)

코드
Num=a_nam_0002      0
Num=a_nam_0041      1
Num=a_nam_829       2
Num=a_nam_0146      3
Num=a_nam_0319      4
                 ... 
Num=a_nam_897     460
Num=a_nam_1069    461
Num=a_nam_1104    462
Num=a_nam_1107    463
Num=a_nam_1198    464
Length: 465, dtype: int64


In [16]:
#추천 함수

def place_REC(코드, cosine_sim=cosine_sim):
    #입력한 코드로 부터 인덱스 가져오기
    idx = indices[코드]

    # 모든 장소에 대해서 해당 장소와의 유사도를 구하기
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 장소들을 정렬
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse = True)

    # 가장 유사한 10개의 장소를 받아옴
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개 장소의 인덱스 받아옴
    place_indices = [i[0] for i in sim_scores]
    
    #기존 데이터에서 해당 인덱스의 값들을 가져온다. 그리고 스코어 열을 추가하여 코사인 유사도도 확인할 수 있게 한다.
    result_place = place.iloc[place_indices].copy()
    result_place['score'] = [i[1] for i in sim_scores]
    
    # 읽어들인 데이터에서 tag부분만 제거, 코드와 스코어만 보이게 함
    del result_place['tag']

    # 가장 유사한 10개의 장소를 반환
    return result_place

In [19]:
place_REC("Num=a_nam_0002")

,Unnamed: 0,이름,코드,태그,score
364,364,김밥천국 정대후문점,Num=a_nam_0166,#먹어야산다#분식#정대후문#포장#혼밥,0.882339
438,438,안암동김밥,Num=a_nam_0139,#먹어야산다#분식#정대후문,0.763456
94,94,절대분식,Num=a_nam_0162,#먹어야산다#분식#한식#백반#정대후문#포장#혼밥,0.731105
445,445,안암동김밥,Num=a_nam_0354,#먹어야산다#분식#정대후문#포장#1인분,0.611106
426,426,엄마네,Num=a_nam_0310,#먹어야산다#분식#법대후문#포장#혼밥,0.580730
170,170,두끼떡볶이 고대점,Num=a_nam_0093,#먹어야산다#분식#참살이길#포장,0.551018
410,410,봉구스밥버거 고려대점,Num=a_nam_0127,#먹어야산다#분식#밥버거#정대후문#포장#혼밥#저렴,0.544376
40,40,동네,Num=a_nam_860,#먹어야산다#한식#정대후문#백반#혼밥#저렴#가성비,0.542599
16,16,동우설렁탕,Num=a_nam_0116,#먹어야산다#한식#설렁탕#정대후문#포장#혼밥#회식#가성비,0.522106
55,55,서울쌈냉면 고대점,Num=a_nam_0120,#먹어야산다#한식#냉면#정대후문#포장#혼밥#저렴#가성비,0.496470
